# Asymetric semantic search
## Bi-encoder (RuBert) Fine-tuning using Sentence-Transformers library

dataset: https://huggingface.co/datasets/sberquad
training overview: https://www.sbert.net/docs/training/overview.html

In [1]:
import torch

device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

mps


In [2]:
from sentence_transformers import SentenceTransformer, models


def raw_bi_encoder():
    word_embedding_model = models.Transformer('cointegrated/rubert-tiny2', max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    bi_encoder = SentenceTransformer(
        modules=[word_embedding_model, pooling_model],
        device=device,
    )

    return bi_encoder

/Users/timoniche/PycharmProjects/NLP/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
bi_encoder = raw_bi_encoder()

print(bi_encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


# Let's make sure it works

In [4]:
from sentence_transformers import evaluation

document_context = [
    'Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.',

    'Скорость света в вакууме — абсолютная величина скорости распространения электромагнитных волн в вакууме. Традиционно обозначается латинской буквой c (произносится как [це]). Скорость света в вакууме — фундаментальная постоянная, не зависящая от выбора инерциальной системы отсчёта (ИСО). Она относится к фундаментальным физическим постоянным, которые характеризуют не просто отдельные тела или поля, а свойства пространства-времени в целом. По современным представлениям, скорость света в вакууме — предельная скорость движения частиц и распространения взаимодействий.',

    'Первый троллейбус был создан в Германии инженером Вернером фон Сименсом, вероятно, под влиянием идеи его брата, проживавшего в Англии доктора Вильгельма Сименса, высказанной 18 мая 1881 года на двадцать втором заседании Королевского научного общества. Электросъём осуществлялся восьмиколёсной тележкой (Kontaktwagen), катившейся по двум параллельным контактным проводам. Провода располагались достаточно близко друг от друга, и при сильном ветре нередко перехлёстывались, что приводило к коротким замыканиям. Экспериментальная троллейбусная линия протяжённостью 540 м (591 ярд), открытая компанией Siemens & Halske в предместье Берлина Галензе (Halensee), действовала с 29 апреля по 13 июня 1882.',
]

question = [
    'На какой период был арендован Россией комплекс Байконур',

    'Как называется абсолютная величина скорости распространения электромагнитных волн в вакууме?',

    'Кем был ослеплен князь Василий Тёмный?',
]

label = [
    1.0,
    1.0,
    0.0,
]

evaluator = evaluation.EmbeddingSimilarityEvaluator(question, document_context, label)
bi_encoder.evaluate(evaluator)

0.8660254037844387

# Evaluate on train-val without Fine-Tuning

In [5]:
from hw4.data_preparer import trainval_shuffled_data

trainval = trainval_shuffled_data(from_file=True)

n_samples = 1000
questions, document_contexts, coss = zip(*trainval[:n_samples])

In [6]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(questions, document_contexts, coss)

In [7]:
bi_encoder = raw_bi_encoder()

In [8]:
bi_encoder.evaluate(evaluator)

0.7866959794370026

# Fine-Tuning

In [9]:
from hw4.data_preparer import trainval_examples

trainval_examples = trainval_examples()[:n_samples]
print(trainval_examples[0].__dict__)

{'guid': '', 'texts': ['что оказала Заметное, хотя и менее значительное влияние на воззрения Локка', 'Заметное, хотя и менее значительное влияние на воззрения Локка оказала психология Гоббса, у которого заимствован, например, порядок изложения Опыта . Описывая процессы сравнения, Локк следует за Гоббсом; вместе с ним он утверждает, что отношения не принадлежат вещам, а составляют результат сравнения, что отношений бесчисленное множество, что более важные отношения суть тождество и различие, равенство и неравенство, сходство и несходство, смежность по пространству и времени, причина и действие. В трактате о языке, то есть в третьей книге Опыта , Локк развивает мысли Гоббса. В учении о воле Локк находится в сильнейшей зависимости от Гоббса; вместе с последним он учит, что стремление к удовольствию есть единственное проходящее через всю нашу психическую жизнь и что понятие о добре и зле у различных людей совершенно различно. В учении о свободе воли Локк вместе с Гоббсом утверждает, что во

In [10]:
from torch.utils.data import DataLoader

trainval_dataloader_set = DataLoader(trainval_examples, shuffle=True, batch_size=32, collate_fn=bi_encoder.smart_batching_collate)

In [11]:
from sentence_transformers import losses

trainval_loss_set = losses.CosineSimilarityLoss(bi_encoder)

In [12]:
trainval_loss_set

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

In [13]:
(query_batch, context_batch), labels = next(iter(trainval_dataloader_set))
query_batch['input_ids'].shape, context_batch['input_ids'].shape, labels.shape

(torch.Size([32, 33]), torch.Size([32, 256]), torch.Size([32]))

In [17]:
bi_encoder.fit(
    train_objectives=[(trainval_dataloader_set, trainval_loss_set)],
    output_path='qa/results',
    epochs=5,
    evaluator=evaluator,
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

In [18]:
bi_encoder.evaluate(evaluator)

0.7925156846997037